In [1]:
#get last week of data for item prices
#2073600 seconds in one week

In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import time
from scipy import stats

In [3]:
def get_prices_lastx(lastx):
    '''Takes latest x time in seconds to retrieve prices for'''
    cutoff = time.time() - lastx
    conn = sqlite3.connect('../geitems.db')
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM prices WHERE timestamp >= {cutoff}""")
    response = cur.fetchall()
    df = pd.DataFrame(response,columns=['id','item_id','timestamp','avgHighPrice','highPriceVolume','avgLowPrice','lowPriceVolume'])
    conn.close()
    return df

In [4]:
time.time() - 2073600

1645804759.9285715

In [5]:
price_df = get_prices_lastx(2073600)

In [6]:
price_df

,id,item_id,timestamp,avgHighPrice,highPriceVolume,avgLowPrice,lowPriceVolume
0,1,9998,1647194100,6750.0,4,5990.0,16
1,2,9998,1647194400,6934.0,31,NaN,0
2,3,9998,1647194700,6998.0,2,5500.0,20
3,4,10010,1646679000,15.0,1,NaN,0
4,5,10029,1646679000,6.0,3,NaN,0
...,...,...,...,...,...,...,...
18139,18140,987,1646679000,9945.0,4,9500.0,30
18140,18141,989,1646679000,20274.0,1,20000.0,26
18141,18142,99,1646679000,9925.0,2,9900.0,326
18142,18143,9978,1646679000,NaN,0,25.0,12


In [7]:
#Make a dictionary with the IQR for each item_id. The key will be the item_id

In [8]:
unique_ids = list(price_df.item_id.unique())

In [9]:
#using avgHighPrice because I think it represents the worst case
#also calc the 50th tile
IQR_dict = {}
for idx in unique_ids:
    item_df = price_df.loc[price_df.item_id == idx]
    avgHighIQR = stats.iqr(item_df.avgHighPrice)
    IQR_dict[f'{idx}'] = (avgHighIQR, item_df.avgHighPrice.quantile())

In [10]:
#first num in IQR, second it 50th tile
# so outside the IQR is 50th tile +- IQR/2
IQR_dict

{'9998': (nan, 6998.0),
 '10010': (nan, 198.0),
 '10029': (4.5, 24.0),
 '10033': (nan, 850.0),
 '10034': (nan, 974.0),
 '10037': (nan, 3995.0),
 '1005': (nan, 96.5),
 '1007': (748.0, 250.0),
 '10077': (0.0, 1.0),
 '10083': (nan, 1543.0),
 '1009': (nan, nan),
 '101': (nan, 879.0),
 '10109': (0.0, 3200.0),
 '1013': (0.0, 101.0),
 '10144': (nan, 1.0),
 '10146': (nan, 50.0),
 '10148': (nan, 614.0),
 '10158': (nan, 113.0),
 '10159': (nan, nan),
 '1017': (nan, 562.0),
 '1019': (nan, 650.0),
 '1021': (nan, 223.0),
 '1027': (nan, 1557.0),
 '10284': (nan, 5024.0),
 '10286': (nan, 54046.0),
 '10292': (nan, 22342.0),
 '10300': (nan, nan),
 '10327': (nan, 88.0),
 '1035': (nan, 1533.0),
 '10352': (0.0, 31597990.0),
 '10368': (nan, 149990.0),
 '10370': (nan, 672884.0),
 '10372': (nan, 818149.0),
 '10374': (nan, 78830.0),
 '10378': (60287.0, 300514.0),
 '1038': (nan, 1946.0),
 '10380': (nan, 601055.0),
 '10384': (0.0, 89356.0),
 '10388': (nan, 706637.0),
 '1044': (nan, 1970.0),
 '10446': (nan, 25935.